In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
import psycopg2  
from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField

In [3]:
bdname='sinesearch_db'
bduser = 'postgres'
bdpassword = 'nef441'
bdhost = '51.79.69.179'
bdport = 5432

db = PostgresqlExtDatabase(bdname, user=bduser, password=bdpassword,
                           host=bdhost, port=bdport)

In [12]:
db.rollback()

In [5]:

from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField

class Users(Model):
    tel_id=IntegerField(index=True)
    name = TextField()
    nicname = TextField(null=True)
    ustatus=IntegerField(default=0)
    age=IntegerField(default=20)
    sex=IntegerField(default=0)
    ganres=JSONField(default={})
    info=JSONField(default={})
    last_visit=DateField(null=True)
    mark_wight=FloatField(default=0)
    just_marked = IntegerField(default=1)

    cms=IntegerField(default=0)
    cfid=IntegerField(default=0)
    cfmes=IntegerField(default=0)
    ctmes=IntegerField(default=0)
    liked=ArrayField(IntegerField,default=[])
    viewed=ArrayField(IntegerField,default=[])
    disliked=ArrayField(IntegerField,default=[])
    predict_films=ArrayField(IntegerField,default=[])
    selections=JSONField(default={})

    class Meta:
        database = db
        db_table='Users'

class Films(Model):
    film_id=IntegerField(unique=True,index=True)
    name = TextField()
    discr = TextField(null=True)
    img = TextField(null=True)
    youtube=TextField(default="")
    stars = FloatField(default=0)
    info=JSONField(default={})
    level=IntegerField(default=0)
    selections=ArrayField(IntegerField,default=[])
    year=IntegerField(default=2000)

    ganres=ArrayField(IntegerField,default=[])
    meanage=FloatField(default=20)
    sex=FloatField(default=0.5)

    likes=IntegerField(default=10)
    dislikes=IntegerField(default=10)
    shit=IntegerField(default=0)
    errors=IntegerField(default=0)
    opening=IntegerField(default=0)
    treilers=IntegerField(default=0)

    class Meta:
        database = db
        db_table='Films'

class Selections(Model):
    name = TextField()
    user_id = IntegerField(default=0)
    stars = FloatField(default=0)
    films = ArrayField(IntegerField,default=[])

    class Meta:
        database = db
        db_table='Selections'

class Messages(Model):
    mes_id = TextField(unique=True)
    text = TextField(default="")
    reply_markup = JSONField(default={})
    user = IntegerField(default=0)
    btime = DateField(null=True)

    class Meta:
        database = db
        db_table='Messages'

class Config(Model):
    name = TextField()
    value = TextField()
    json=JSONField(default={})

    class Meta:
        database = db
        db_table='Config'

class Dataset(Model):
    user_value = JSONField()
    film_value = JSONField()
    result = IntegerField(default=0)
    data = DateField()

    class Meta:
        database = db
        db_table='Dataset'

In [15]:
Films.select().where(Films.stars>9).count()

7968

In [13]:
Films.update({Films.stars:(3.5*Films.dislikes+1*Films.shit+10*Films.likes)/(Films.likes + Films.shit + Films.dislikes)}).where(Films.stars>10).execute()

1134

In [8]:
with open("collections","r") as f:
    collections=json.load(f)

In [ ]:
col=[]
for i , c in tqdm(enumerate(collections)):
    col.append({
         "name":c,
         "films":[int(k) for k in collections[c]]
     })
    if i % 100 ==0:
        Selections.insert_many(col).execute()
        col=[]
Selections.insert_many(col).execute()

In [10]:
fms=[]
db.rollback()
with open("films.jsonlines","r") as f:
    for k, line in tqdm(enumerate(f.readlines())):
        film = json.loads(line)
        info=film.copy()
        if "Жанр" not in film:
            film["Жанр"]=""
        
        
        comp_film={
            "name":info["name"],
            "discr":info["discr"],
            "img":info["img"],
            "youtube":info["youtube"],
            "film_id":info["film_id"],   
            "stars":0,
            "ganres":[gange.index(g) for g in film["Жанр"].replace(" ","").replace(".","").split(",")]
        }
        try:
            comp_film["year"]=int(re.search(r"\(\d+\)",info["name"])[0][1:-1])
            comp_film["name"]=comp_film["name"].replace(re.search(r"\(\d+\)",info["name"])[0],"")
        except:
            print(comp_film["name"])
            comp_film["year"]=2000
#             break
            
        if "Рейтинг" in info:
            comp_film["stars"] = float(info["Рейтинг"].replace(" ","").replace(",",".").replace("%",""))/10
        del info["name"]
        del info["discr"]
        del info["img"]
        del info["youtube"]
        del info["film_id"]
#         info["Год выпуска"]=comp_film["year"]
        del info["data"]

        
        comp_film["selections"]=[]
        comp_film["level"]=4
        
            
        comp_film["info"]=info
#         break
        
        fms.append(comp_film)
        if k % 500 == 0:
            Films.insert_many(fms).on_conflict_ignore().execute()   
            fms=[]
Films.insert_many(fms).on_conflict_ignore().execute()

In [ ]:
len(Films.select().where(Films.level==1).count().execute())

In [ ]:
with open("films","w") as f:
    json.dump(fms,f)

**Ganre**

In [8]:
with open("gange","r") as f:
    gange = json.load(f)

In [6]:
import torch
import os
import json
import time
from tqdm import tqdm_notebook as tqdm

In [9]:
with open("gange","r") as f:
    gange = json.load(f)
with open("vocab","r") as f:
    vocab = json.load(f)
    
EMBED_DIM = 16
NUN_CLASS = 1 #len(gange)
NGRAMS = 2
BATCH_SIZE = 16
VOCAB_SIZE=len(vocab)
N_EPOCHS =5 

model = TextGanre(BATCH_SIZE, VOCAB_SIZE, EMBED_DIM, NUN_CLASS)

model.load_state_dict(torch.load("model_lstm_short"))


RuntimeError: Error(s) in loading state_dict for TextGanre:
	Missing key(s) in state_dict: "embedding.weight", "basic_rnn.weight_ih_l0", "basic_rnn.weight_hh_l0", "basic_rnn.bias_ih_l0", "basic_rnn.bias_hh_l0", "clasifiter.0.weight", "clasifiter.0.bias", "clasifiter.2.weight", "clasifiter.2.bias". 
	Unexpected key(s) in state_dict: "0.weight", "1.weight_ih_l0", "1.weight_hh_l0", "1.bias_ih_l0", "1.bias_hh_l0". 

In [8]:
import re

class Porter:
	PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
	REFLEXIVE = re.compile(u"(с[яь])$")
	ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
	PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
	VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
	NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
	RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
	DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
	DER = re.compile(u"ость?$")
	SUPERLATIVE = re.compile(u"(ейше|ейш)$")
	I = re.compile(u"и$")
	P = re.compile(u"ь$")
	NN = re.compile(u"нн$")

	def stem(word):
		word = word.lower()
		word = word.replace(u'ё', u'е')
		m = re.match(Porter.RVRE, word)
		if m.groups():
			pre = m.group(1)
			rv = m.group(2)
			temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
			if temp == rv:
				rv = Porter.REFLEXIVE.sub('', rv, 1)
				temp = Porter.ADJECTIVE.sub('', rv, 1)
				if temp != rv:
					rv = temp
					rv = Porter.PARTICIPLE.sub('', rv, 1)
				else:
					temp = Porter.VERB.sub('', rv, 1)
					if temp == rv:
						rv = Porter.NOUN.sub('', rv, 1)
					else:
						rv = temp
			else:
				rv = temp
			
			rv = Porter.I.sub('', rv, 1)

			if re.match(Porter.DERIVATIONAL, rv):
				rv = Porter.DER.sub('', rv, 1)

			temp = Porter.P.sub('', rv, 1)
			if temp == rv:
				rv = Porter.SUPERLATIVE.sub('', rv, 1)
				rv = Porter.NN.sub(u'н', rv, 1)
			else:
				rv = temp
			word = pre+rv
		return word
	stem=staticmethod(stem)

In [9]:
def text_preproc(text):
    reg = re.compile('[^а-яА-Я ]')
    text = (reg.sub('', text))
    text=re.compile("\s+").sub(" ",text).lower()
    
    res=[]
    for w in text.split(" "):
        try:
            if len(w)>4:
                try:
                    res.append(vocab[Porter.stem(w)])
                except:
                    pass
            else:
                res.append(vocab[w])
        except:
            pass
    
    return res

In [10]:
text_preproc("Очень страшный фильм")

[16206, 62696, 23914]

In [12]:
def generate_batch(batch):
    d=[b["ganre"][0] for b in batch]
    label = torch.tensor(d).to(torch.float)
    text = [torch.tensor(b["discr"]).to(torch.int64) for b in batch]
    
    text=torch.nn.utils.rnn.pad_sequence( text , batch_first=True)
    return text.view(-1,len(batch)), label

In [13]:
def predict(text):
    text, _ = generate_batch([{
        "discr":text_preproc(text),
        "ganre":[0]
    }])
    return text

test = predict("Там, где круглый год лежат снега, а небо озаряет северное сияние, живет Ледяная принцесса Лилли с голубыми, как древний лед, волосами. Однажды Лилли и её друг, полярный медведь Лимбо, встречают юного дракона, который не умеет извергать пламя. Лилли обещает помочь ему. Но путь, который их ждет, будет не из легких, впереди новых друзей ждет немало приключений и опасных испытаний.")

In [ ]:
model.load_state_dict(torch.load(PATH))

In [18]:
model.h_0 = torch.zeros(1,1, 128)
model.c_0 = torch.zeros(1,1, 128)
model.eval()
pred=model(test)

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'